# A : nodige modules

In [1]:
# standaard in python
import os 
import datetime
import sys
import json

In [2]:
# te installeren packages en modules
import numpy as np
import pandas as pd
import matplotlib as plt

In [3]:
# eigen packages modules
current_working_directory = os.getcwd()                                    
parent_working_directory = os.path.dirname(current_working_directory)      
scripts_directory = os.path.join(parent_working_directory, 'scripts')      
sys.path.append(scripts_directory)                                         
# sys.path.remove(scripts_directory)
# print(sys.path)

import comeinhandy as cih
import simulatie as sim

# B : functies om herhaling te vermijden

In [4]:
#  zie cih = 'comeinhandy' module

# C : opladen van de gegevens

In [5]:
# C.1 opladen JSON file met MSR waarden (min_sust_rate = minimu sustainable rate)
min_sust_rate_locatie = "../data/raw/productiemodel/data_productie/master_data.json"
df_msr_gegevens = cih.laad_msr_gegevens_in_dataframe(min_sust_rate_locatie)
df_msr_gegevens

,factory,msr_description,msr_value
0,BRU,BRU msr,1244
1,STO,STO msr,274


In [6]:
# C.2 bepaal waar de dagelijk productie sets (per fabriek) zich bevinden op schijf
dagelijkse_productie_locatie = "../data/raw/productiemodel/data_productie/daily_production/"

In [7]:

# C.2.1. Laad dagelijks productie voor BRU in zijn dataframe
factory = 'BRU'
dagelijkse_productie_BRU = cih.laad_dagelijkse_productie_in_dataframe(
                                factory, 
                                os.path.join(dagelijkse_productie_locatie, factory))
# dagelijkse_productie_BRU

# C.2.2 voeg MSR waarden toe op het BRU dagelijkse productie dataframe
dagelijkse_productie_BRU = dagelijkse_productie_BRU.merge(
                                        df_msr_gegevens, on='factory', how='left')
dagelijkse_productie_BRU

# C.2.3 verwijder records waar onderhoud werd gepland, zie opgave beschrijving 5, bullet 1 en dixit Tim in de klas
dagelijkse_productie_BRU = dagelijkse_productie_BRU[dagelijkse_productie_BRU['maintenance'] == 'No']
dagelijkse_productie_BRU

,DoW,hour,minute,date,maintenance,prod_loss,prod_loss_perc,production,factory,msr_description,msr_value
0,Monday,1,0,01-01-2018 00:00:00.0000,No,61,5,1183,BRU,BRU msr,1244
1,Tuesday,1,0,01-02-2018 00:00:00.0000,No,106,9,1138,BRU,BRU msr,1244
2,Wednesday,1,0,01-03-2018 00:00:00.0000,No,129,10,1115,BRU,BRU msr,1244
3,Thursday,1,0,01-04-2018 00:00:00.0000,No,278,22,966,BRU,BRU msr,1244
4,Friday,1,0,01-05-2018 00:00:00.0000,No,218,17,1026,BRU,BRU msr,1244
...,...,...,...,...,...,...,...,...,...,...,...
1825,Saturday,1,0,12-31-2022 00:00:00.0000,No,203,16,1041,BRU,BRU msr,1244
1826,Sunday,1,0,01-01-2023 00:00:00.0000,No,231,19,1013,BRU,BRU msr,1244
1827,Monday,1,0,01-02-2023 00:00:00.0000,No,181,15,1063,BRU,BRU msr,1244
1828,Tuesday,1,0,01-03-2023 00:00:00.0000,No,198,16,1046,BRU,BRU msr,1244


In [8]:
# C.3.1. Laad dagelijks productie voor STO in zijn dataframe
factory = 'STO'
dagelijkse_productie_STO = cih.laad_dagelijkse_productie_in_dataframe(
                                factory, 
                                os.path.join(dagelijkse_productie_locatie, factory))
# dagelijkse_productie_STO

# C.2.2 voeg MSR waarden toe op het STO dagelijkse productie dataframe
dagelijkse_productie_STO = dagelijkse_productie_STO.merge(
                                        df_msr_gegevens, on='factory', how='left')
# dagelijkse_productie_STO

# C.3.3 verwijder records waar onderhoud werd gepland, zie opgave beschrijving 5, bullet 1 en dixit Tim in de klas
dagelijkse_productie_STO = dagelijkse_productie_STO[dagelijkse_productie_STO['maintenance'] == 'No']
dagelijkse_productie_STO

,DoW,hour,minute,date,maintenance,prod_loss,prod_loss_perc,production,factory,msr_description,msr_value
0,Monday,1,0,01-01-2018 00:00:00.0000,No,9,3,265,STO,STO msr,274
1,Tuesday,1,0,01-02-2018 00:00:00.0000,No,9,3,265,STO,STO msr,274
2,Wednesday,1,0,01-03-2018 00:00:00.0000,No,18,6,256,STO,STO msr,274
3,Thursday,1,0,01-04-2018 00:00:00.0000,No,22,8,252,STO,STO msr,274
4,Friday,1,0,01-05-2018 00:00:00.0000,No,31,11,243,STO,STO msr,274
...,...,...,...,...,...,...,...,...,...,...,...
1825,Saturday,1,0,12-31-2022 00:00:00.0000,No,51,19,223,STO,STO msr,274
1826,Sunday,1,0,01-01-2023 00:00:00.0000,No,53,20,221,STO,STO msr,274
1827,Monday,1,0,01-02-2023 00:00:00.0000,No,61,22,213,STO,STO msr,274
1828,Tuesday,1,0,01-03-2023 00:00:00.0000,No,70,25,204,STO,STO msr,274


In [9]:
# C.5.0 Breng alle gegevens te samen in een groot dataframe voor analyse
dagelijkse_productie_ALL = pd.concat(
                            [dagelijkse_productie_BRU, 
                             dagelijkse_productie_STO],
                               ignore_index=True)
#  save het voor analyse in een Excel sheet
dagelijkse_productie_ALL.to_excel("../data/interim/dagelijkse_productie_ALL.xlsx")
dagelijkse_productie_ALL

,DoW,hour,minute,date,maintenance,prod_loss,prod_loss_perc,production,factory,msr_description,msr_value
0,Monday,1,0,01-01-2018 00:00:00.0000,No,61,5,1183,BRU,BRU msr,1244
1,Tuesday,1,0,01-02-2018 00:00:00.0000,No,106,9,1138,BRU,BRU msr,1244
2,Wednesday,1,0,01-03-2018 00:00:00.0000,No,129,10,1115,BRU,BRU msr,1244
3,Thursday,1,0,01-04-2018 00:00:00.0000,No,278,22,966,BRU,BRU msr,1244
4,Friday,1,0,01-05-2018 00:00:00.0000,No,218,17,1026,BRU,BRU msr,1244
...,...,...,...,...,...,...,...,...,...,...,...
3447,Saturday,1,0,12-31-2022 00:00:00.0000,No,51,19,223,STO,STO msr,274
3448,Sunday,1,0,01-01-2023 00:00:00.0000,No,53,20,221,STO,STO msr,274
3449,Monday,1,0,01-02-2023 00:00:00.0000,No,61,22,213,STO,STO msr,274
3450,Tuesday,1,0,01-03-2023 00:00:00.0000,No,70,25,204,STO,STO msr,274


In [10]:
dagelijkse_productie_ALL['production'].mean()
# dagelijkse_productie_ALL['production'].median()
# dagelijkse_productie_ALL.describe(include='all')

497.17352259559675

# D : minimale controle van het Dataframe

In [11]:
dagelijkse_productie_ALL

,DoW,hour,minute,date,maintenance,prod_loss,prod_loss_perc,production,factory,msr_description,msr_value
0,Monday,1,0,01-01-2018 00:00:00.0000,No,61,5,1183,BRU,BRU msr,1244
1,Tuesday,1,0,01-02-2018 00:00:00.0000,No,106,9,1138,BRU,BRU msr,1244
2,Wednesday,1,0,01-03-2018 00:00:00.0000,No,129,10,1115,BRU,BRU msr,1244
3,Thursday,1,0,01-04-2018 00:00:00.0000,No,278,22,966,BRU,BRU msr,1244
4,Friday,1,0,01-05-2018 00:00:00.0000,No,218,17,1026,BRU,BRU msr,1244
...,...,...,...,...,...,...,...,...,...,...,...
3447,Saturday,1,0,12-31-2022 00:00:00.0000,No,51,19,223,STO,STO msr,274
3448,Sunday,1,0,01-01-2023 00:00:00.0000,No,53,20,221,STO,STO msr,274
3449,Monday,1,0,01-02-2023 00:00:00.0000,No,61,22,213,STO,STO msr,274
3450,Tuesday,1,0,01-03-2023 00:00:00.0000,No,70,25,204,STO,STO msr,274


In [12]:
dagelijkse_productie_ALL.shape

(3452, 11)

In [13]:
dagelijkse_productie_ALL.columns

Index(['DoW', 'hour', 'minute', 'date', 'maintenance', 'prod_loss',
       'prod_loss_perc', 'production', 'factory', 'msr_description',
       'msr_value'],
      dtype='object')

In [14]:
dagelijkse_productie_ALL.index

RangeIndex(start=0, stop=3452, step=1)

In [15]:
dagelijkse_productie_ALL.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3452 entries, 0 to 3451
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   DoW              3452 non-null   object
 1   hour             3452 non-null   int64 
 2   minute           3452 non-null   int64 
 3   date             3452 non-null   object
 4   maintenance      3452 non-null   object
 5   prod_loss        3452 non-null   object
 6   prod_loss_perc   3452 non-null   object
 7   production       3452 non-null   object
 8   factory          3452 non-null   object
 9   msr_description  3452 non-null   object
 10  msr_value        3452 non-null   int64 
dtypes: int64(3), object(8)
memory usage: 296.8+ KB


In [16]:
dagelijkse_productie_ALL.describe()
dagelijkse_productie_ALL.describe(include = "all") 

,DoW,hour,minute,date,maintenance,prod_loss,prod_loss_perc,production,factory,msr_description,msr_value
count,3452,3452.0,3452.0,3452,3452,3452.0,3452.0,3452.0,3452,3452,3452.000000
unique,7,NaN,NaN,1830,1,433.0,93.0,580.0,2,2,NaN
top,Monday,NaN,NaN,01-01-2018 00:00:00.0000,No,1244.0,100.0,0.0,BRU,BRU msr,NaN
freq,496,NaN,NaN,2,3452,308.0,434.0,434.0,1736,1736,NaN
mean,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,761.809965
std,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,485.062123
min,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,274.000000
25%,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,274.000000
50%,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1244.000000
75%,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1244.000000


In [17]:
cih.show_info_about_column(dagelijkse_productie_BRU, 'production')

----- None -- production -----------------------------------
df[col].describe(include='all') = count     1736
unique     387
top          0
freq       308
Name: production, dtype: int64 

----- None -- production -----------------------------------
df[col].nunique(dropna=True) = 387 
 
----- None -- production -----------------------------------
df[col].value_counts() = production
0       308
1011     13
1115     12
1005     11
1019     11
       ... 
869       1
886       1
874       1
418       1
876       1
Name: count, Length: 387, dtype: int64 
 
----- end of query --------------------------------------------------


In [18]:
cih.show_info_about_column(dagelijkse_productie_STO, 'production')

----- None -- production -----------------------------------
df[col].describe(include='all') = count     1716
unique     194
top          0
freq       126
Name: production, dtype: int64 

----- None -- production -----------------------------------
df[col].nunique(dropna=True) = 194 
 
----- None -- production -----------------------------------
df[col].value_counts() = production
0      126
202     55
198     51
203     50
209     50
      ... 
110      1
34       1
145      1
41       1
152      1
Name: count, Length: 194, dtype: int64 
 
----- end of query --------------------------------------------------


In [19]:
cih.show_info_about_column(dagelijkse_productie_ALL, 'production')

----- None -- production -----------------------------------
df[col].describe(include='all') = count     3452
unique     580
top          0
freq       434
Name: production, dtype: int64 

----- None -- production -----------------------------------
df[col].nunique(dropna=True) = 580 
 
----- None -- production -----------------------------------
df[col].value_counts() = production
0      434
202     55
198     51
203     50
209     50
      ... 
474      1
630      1
642      1
896      1
152      1
Name: count, Length: 580, dtype: int64 
 
----- end of query --------------------------------------------------


# E : permanente cleaning van de data

In [20]:
# dagelijkse_productie_STO.info()

# # columns_to_convert_to_int = ['prod_loss', 'production']
# # dagelijkse_productie_STO[columns_to_convert_to_int] = dagelijkse_productie_STO[columns_to_convert_to_int].astype(int) 
# # dagelijkse_productie_STO['prod_loss'] = dagelijkse_productie_STO['prod_loss'].astype(int) 
# # dagelijkse_productie_STO['production'] = dagelijkse_productie_STO['production'].astype(int) 
# dagelijkse_productie_STO['prod_loss'] = pd.to_numeric(dagelijkse_productie_STO['prod_loss'], errors='coerce')

# # columns_to_convert_to_float = ['prod_loss_perc']
# # dagelijkse_productie_STO[columns_to_convert_to_float] = dagelijkse_productie_STO[columns_to_convert_to_float].astype(float) 

# dagelijkse_productie_STO.info()

In [21]:
# C.3.1.b identificeer de 0 dagen productie lijnen
dagelijkse_productie_BRU_geen_productie = dagelijkse_productie_BRU[dagelijkse_productie_BRU['maintenance'] == 'No']
# dagelijkse_productie_BRU_maintenance_removed


# F : analyse van de mogelijke verdelingen

# G : uitvoeren van de Simulatie

In [23]:
xx = np.linspace(0, 10 ** 4)
data = sim.simulatie(7)
Fxx = sim.cumulatieve_distributie_functie(xx)
# plt.plot(xx, Fxx)